In [1]:
import dgl
import time
import torch as th
import numpy as np
import torch.nn.functional as F

from load_data import load_ogb
from model import SAGE, GAT, get_eigen
from train_eval import train_ft_embed

Using backend: pytorch


This notebook shows the performance of GraphSage on the OGBN-products graph with trainable embeddings on the nodes. We use scaled eigenvectors to initialize the node embeddings. The logic here is that if eigenvectors can serve as positional node embeddings, fine-tuning the node embeddings should give us even better performance. However, in this experiment, we tune the node embeddings with the same learning rate as the one for the GraphSage model.

In [3]:
g, num_labels = load_ogb('ogbn-products')
g.ndata.pop('features')

in_feats = 128
eigen_vals, eigen_vecs = get_eigen(g, in_feats, 'ogbn-products')
nfeat = th.tensor(eigen_vecs * np.sqrt(eigen_vals).reshape((1, len(eigen_vals))), dtype=th.float32)
labels = g.ndata.pop('labels')

hyperparams = {
    'batch_size': 1024,
    'num_workers': 0,
    'num_hidden': 256,
    'num_layers': 3,
    'dropout': 0.5,
    'lr': 0.003,
    'sparse_lr': 0.003,
    'num_epochs': 20,
    'fanouts': [15,10,5],
    'eval_batch_size': 10000
}
device = th.device('cuda:2')

# Pack data
data = g, nfeat, labels
# Define model
model = SAGE(in_feats, hyperparams['num_hidden'], num_labels,
             hyperparams['num_layers'], F.relu, hyperparams['dropout'])
train_ft_embed(model, data, hyperparams, device, 1)

load ogbn-products
finish loading ogbn-products
finish constructing ogbn-products
embedding is in  cuda:2


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00000 | Step 00000 | Loss 3.8519 | Train Acc 0.0205 | Speed (samples/sec) nan | GPU 4999.9 MB
Epoch 00000 | Step 00020 | Loss 1.0807 | Train Acc 0.7715 | Speed (samples/sec) 442.8895 | GPU 5001.3 MB
Epoch 00000 | Step 00040 | Loss 0.5557 | Train Acc 0.8838 | Speed (samples/sec) 444.9740 | GPU 5010.4 MB
Epoch 00000 | Step 00060 | Loss 0.4304 | Train Acc 0.8984 | Speed (samples/sec) 438.2583 | GPU 5010.4 MB
Epoch 00000 | Step 00080 | Loss 0.3049 | Train Acc 0.9160 | Speed (samples/sec) 437.0898 | GPU 5010.4 MB
Epoch 00000 | Step 00100 | Loss 0.3824 | Train Acc 0.9033 | Speed (samples/sec) 440.6758 | GPU 5011.1 MB
Epoch 00000 | Step 00120 | Loss 0.3783 | Train Acc 0.9102 | Speed (samples/sec) 438.4916 | GPU 5011.1 MB
Epoch 00000 | Step 00140 | Loss 0.2801 | Train Acc 0.9287 | Speed (samples/sec) 434.6644 | GPU 5011.1 MB
Epoch 00000 | Step 00160 | Loss 0.2674 | Train Acc 0.9395 | Speed (samples/sec) 435.9317 | GPU 5011.1 MB
Epoch 00000 | Step 00180 | Loss 0.3101 | Train Acc 0.9238 | 

  0%|          | 0/245 [00:00<?, ?it/s]

Epoch Time(s): 405.7057


...

Val acc 0.9044, Test Acc: 0.7779
Epoch 00019 | Step 00000 | Loss 0.0056 | Train Acc 0.9980 | Speed (samples/sec) 380.2605 | GPU 5020.5 MB
Epoch 00019 | Step 00020 | Loss 0.0268 | Train Acc 0.9971 | Speed (samples/sec) 380.3086 | GPU 5020.5 MB
Epoch 00019 | Step 00040 | Loss 0.0195 | Train Acc 0.9941 | Speed (samples/sec) 380.3611 | GPU 5020.5 MB
Epoch 00019 | Step 00060 | Loss 0.0251 | Train Acc 0.9941 | Speed (samples/sec) 380.4189 | GPU 5020.5 MB
Epoch 00019 | Step 00080 | Loss 0.0031 | Train Acc 0.9990 | Speed (samples/sec) 380.4356 | GPU 5020.5 MB
Epoch 00019 | Step 00100 | Loss 0.0305 | Train Acc 0.9951 | Speed (samples/sec) 380.5671 | GPU 5020.5 MB
Epoch 00019 | Step 00120 | Loss 0.0307 | Train Acc 0.9941 | Speed (samples/sec) 380.6558 | GPU 5020.5 MB
Epoch 00019 | Step 00140 | Loss 0.0012 | Train Acc 1.0000 | Speed (samples/sec) 380.7576 | GPU 5020.5 MB
Epoch 00019 | Step 00160 | Loss 0.0211 | Train Acc 0.9971 | Speed (samples/sec) 380.8744 | GPU 5020.5 MB
Epoch 00019 | Step 001

  0%|          | 0/245 [00:00<?, ?it/s]

Epoch Time(s): 500.6559


100%|██████████| 245/245 [01:05<00:00,  3.76it/s]


Val acc 0.9035, Test Acc: 0.7779
Avg epoch time: 662.7364188671112
Best val acc 0.9148, best test acc: 0.7898
